In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch_geometric.nn.pool import global_mean_pool

In [2]:
class GCNConvByHand(nn.Module):
    """maps D x N to D x N"""
    def __init__(self, dim_in):
        super().__init__()
        self.linear = torch.nn.Linear(dim_in, dim_in, bias=True)

    def forward(self, x, A):
        num_nodes = A.shape[0]
        omega_k = self.linear.weight
        beta_k = self.linear.bias.reshape(-1, 1)
        # print("omega_k", omega_k, "beta_k", beta_k)
        H_k = x
        x = torch.matmul(
            beta_k, torch.reshape(torch.ones(num_nodes), (1, -1))
        ) + torch.matmul(omega_k, torch.matmul(H_k, A + torch.eye(num_nodes)))
        return x

In [ ]:
class GCNByHand(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConvByHand(in_channels)
        self.conv2 = GCNConvByHand(in_channels)
        self.out = torch.nn.Linear(in_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        # print("x after first layer\n", x)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        # print("x after second layer\n", x)
        x = torch.relu(x)
        x = torch.transpose(x, 0, 1)
        x = self.out(x)
        x = torch.sigmoid(x)
        # print("x after output layer\n", x)
        # x = global_mean_pool(x, torch.zeros(x.size(1)).long())  # all are from graph 0, single graph
        return x

In [ ]:
num_nodes = 3       # N
num_features = 2    # D
num_labels = 1      # O

# the following needs to be replaced by some node embeddings
x = torch.randn((num_features, num_nodes)) # D x N

# print("x", x, x.shape)

A = torch.tensor([[0, 1, 1], [1, 0, 0], [1, 0, 0]])  # adjacency matrix ( N x N )

model = GCNByHand(in_channels=num_features, out_channels=num_labels)

out = model(x, A)   # N x O

print("output")
print(out)

output
tensor([[0.6354],
        [0.6309],
        [0.6312]], grad_fn=<SigmoidBackward0>)
